In [29]:
#Imports
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
import pandas as pd
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)
import os
from dotenv import load_dotenv
load_dotenv()
import requests
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from pathlib import Path
%matplotlib inline
import alpaca_trade_api as tradeapi
#import plotly.plotly as py
import plotly.graph_objs as go

In [30]:
csvpath = Path("Resources/BTC.csv")
BTC = pd.read_csv(csvpath)

In [31]:
# Setting axis and cleaning data


In [32]:
#Establishing test and train data
close_data = BTC['Value'].values
close_data = close_data.reshape((-1,1))

In [33]:
split_percent = 0.80
split = int(split_percent*len(close_data))

In [34]:
close_train = close_data[:split]
close_test = close_data[split:]

In [35]:
date_train = BTC['Date'][:split]
date_test = BTC['Date'][split:]
print(len(close_train))
print(len(close_test))

2839
710


In [51]:
look_back = 90

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [52]:
#Build Model
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')





In [53]:
# Show the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10)                480       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________


In [54]:
# Train the model
num_epochs = 10
model.fit_generator(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/10


ValueError: in user code:

    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:788 run_step  **
        outputs = m

In [55]:
prediction = model.predict_generator(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "BTC",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

ValueError: in user code:

    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\lwbak\.conda\lib\site-packages\tensorflow\python\keras\engine\training.py:1461 run_step  **
        output

In [43]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = BTC['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)

In [44]:
forecast

array([ 3.91584700e+04,  2.42781763e+03,  1.90710918e+04,  1.26255811e+03,
       -3.77502563e+02,  7.51584864e+00,  3.97086334e+02, -4.52343369e+01,
        4.23224854e+02, -4.90091667e+01,  4.49306183e+02, -5.20901222e+01,
        4.76633026e+02, -5.53802223e+01,  5.05996796e+02, -5.88370361e+01,
        5.37177124e+02,  1.01902634e+02, -3.76342916e+00,  2.99437592e+02,
       -5.94433975e+01,  5.79698853e+02, -6.74259720e+01,  6.15372986e+02,
       -7.16230621e+01,  6.53222717e+02, -7.59361572e+01,  6.92692383e+02,
       -8.07101517e+01,  7.34989441e+02, -8.55408707e+01])